In [1]:
import numpy as np
import time

In [2]:
def load_data(h,size):
    if h == 1:
        data = np.loadtxt('h'+str(size)+'.txt')
    elif h == 0:
        data = np.loadtxt('l'+str(size)+'.txt')
    else:
        print('Error.')
    proftis = data[:,3]
    data = data[:,1:3]
    return data,proftis

In [3]:
data = None
profits = None
disMat = None
inner_epochs =1000
meta_epochs = 2000
size = None

In [4]:
def modification(tour):
    #return np.random.permutation(tour)
    return tour
def initial_tour(size):
    l = int(size*0.75)
    tour = np.zeros((l+1))
    tour[0] = 0
    tour[l] = 0
    tour[1:l] = np.random.choice(size-1,l-1,replace=False)+1
    return tour
def tour_cost(tour):
    global disMat,profits
    tour_length = len(tour)
    cost = 0
    for i in range(tour_length-1):
        cost += disMat[int(tour[i]),int(tour[i+1])]
        cost -= profits[i]
    return cost
def simulated_annealing(tour,epochs,cost):
    length = len(tour)
    Tinit = 5
    Tfinal = 1
    for k in range(epochs):
        T = np.power(Tinit*(Tfinal/Tinit),k/epochs)
        choices = np.random.choice(length-2,2,replace=False);
        choices = choices +1
        i = min(choices)
        j = max(choices)
        new_tour = np.copy(tour);
        new_tour[i:j+1] = tour[i:j+1][::-1];
        new_cost = tour_cost(new_tour)
        delta = new_cost - cost
        p = min(np.exp(-delta/T),1)
        if np.random.rand() < p:
            tour = np.copy(new_tour)
            cost = new_cost
    return np.copy(tour),cost
def insert(tour):
    length = len(tour)
    if length == size+1:
        return simulated_annealing(tour,500,tour_cost(tour))
    else:
        j = np.random.choice(np.setdiff1d(range(1,size),tour),1)
        new_tour = np.zeros(length+1)
        shuffle = np.copy(tour[1:length-1])
        j_ = np.random.randint(len(shuffle))
        shuffle = np.insert(shuffle,j_,j)
        new_tour[1:length] = modification(shuffle)
        new_cost = tour_cost(new_tour)
        return simulated_annealing(new_tour,inner_epochs,new_cost)
def remove(tour):
    length = len(tour)
    #j = np.random.choice(length-2,1)+1
    can = profits[tour[1:length-1]]
    j = np.argmin(can)
    new_tour = np.delete(tour,j+1)
    new_tour[1:length-2] = modification(new_tour[1:length-2])
    new_cost = tour_cost(new_tour)
    return simulated_annealing(new_tour,200,new_cost)

In [5]:
def meta_sa(s):
    global profits,data,disMat,size
    size = s
    data,profits = load_data(1,size)
    disMat = np.zeros((size,size))
    best_tour = None
    best_cost = 0
    for i in range(size):
        for j in range(size):
            disMat[i,j] = np.linalg.norm(data[i,:] - data[j,:])
    tour = initial_tour(size)
    cost = tour_cost(tour)
    tour,cost = simulated_annealing(tour,200,cost)
    length = len(tour)
    print(cost,length)
    Tinit = 10
    Tfinal = 1
    for k in range(1,meta_epochs):
        if k % 100 == 0:
            print(k,cost,len(tour))
        T = np.power(Tinit*(Tfinal/Tinit),k/meta_epochs)
        p = np.random.rand() 
        if p > 0.66:
            new_tour,new_cost = insert(tour)
        elif 0.66 < p and 0.33 < p:
            new_tour,new_cost = remove(tour)
        else:
            new_tour,new_cost = simulated_annealing(tour,1000,cost)
        delta = new_cost - cost
        p = min(np.exp(-delta/T),1)
        if np.random.rand() < p:
            tour = np.copy(new_tour)
            cost = new_cost
        if cost < best_cost:
            best_tour = tour
            best_cost = cost
    return best_tour

In [6]:
start = time.time()
tour_1 = meta_sa(51)
cpu_usage = (time.time()-start)
cost_1 = tour_cost(tour_1)
print(cost_1,len(tour_1),cpu_usage)
print(tour_1)

-201.18454879270854 39
100 -685.0219283349559 52
200 -692.1616471333784 52
300 -691.8225500800082 52
400 -697.1038278044698 52
500 -698.5931915564274 52
600 -698.3990706972731 52
700 -699.0185286978301 52
800 -696.8077943041044 52
900 -699.3757785151831 52
1000 -698.1453406852139 52
1100 -697.8850284919546 52
1200 -700.331609676424 52
1300 -700.442944710775 52
1400 -701.3083174943336 52
1500 -701.3083174943336 52
1600 -701.554500302125 52
1700 -701.5545003021255 52
1800 -701.5545003021255 52
1900 -699.8137398312015 52
-701.5545003021255 52 124.87832164764404
[ 0. 21.  7. 25. 30. 27.  2. 35. 34. 19. 28.  1. 15. 49. 20. 33. 29.  9.
 38. 32. 44. 14. 43. 41. 18. 39. 40. 12. 24. 13. 23. 42.  6. 22. 47.  5.
 26. 50. 45. 11. 46. 17.  3. 16. 36.  4. 48.  8. 37. 10. 31.  0.]


In [7]:
start = time.time()
tour_2 = meta_sa(76)
cpu_usage = (time.time()-start)
cost_2 = tour_cost(tour_2)
print(cost_2,len(tour_2),cpu_usage)
print(tour_2)

-256.6767286654101 58
100 -1100.0091634274092 71
200 -1219.7449260585736 77
300 -1220.6302795369177 77
400 -1222.5473597419825 77
500 -1229.202247872615 77
600 -1228.5009684013073 77
700 -1228.5950158530595 77
800 -1230.3596421934606 77
900 -1230.3816412601466 77
1000 -1231.1845328190714 77
1100 -1230.6212989328876 77
1200 -1230.3926753961587 77
1300 -1230.6024666429446 77
1400 -1230.5992998662014 77
1500 -1230.7169343991848 77
1600 -1231.3750371460467 77
1700 -1230.4339613826548 77
1800 -1228.118002542319 77
1900 -1231.3970362127327 77
-1231.3970362127327 77 154.4610617160797
[ 0. 32. 62. 15. 50. 16. 39. 11. 25. 66. 75. 74.  3. 67.  5.  1. 73. 29.
 44. 28. 47. 46. 20. 68. 35. 70. 59. 69. 19. 36.  4. 14. 56. 12. 26. 51.
 33. 45.  7. 53. 18. 34.  6. 52. 13. 58. 10. 65. 64. 37. 30.  9. 57. 71.
 38.  8. 24. 54. 17. 49. 31. 43.  2. 23. 48. 22. 55. 40. 42. 41. 63. 21.
 60. 27. 61. 72.  0.]


In [ ]:
start = time.time()
tour_3 = meta_sa(101)
cpu_usage = (time.time()-start)
cost_3 = tour_cost(tour_3)
print(cost_3,len(tour_3),cpu_usage)
print(tour_3)

-163.8198474978656 76
100 -1287.558862365599 82
200 -1303.3225782953068 83
300 -1340.74903874781 86
400 -1405.215916152828 91
500 -1415.6917700834692 93
600 -1603.616444603329 102
700 -1612.7978383309683 102
800 -1614.437736612575 102
900 -1617.0964154743015 102
1000 -1612.3235894097918 102


In [ ]:
len(tour_2)

In [ ]:
len(tour_3)

In [ ]:
profits[[1,2,4]]
